# Audio TTS Datasets

> TTS datasets

In [ ]:
#| default_exp audio.datasets.tts

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| export
import torch
from torch.utils.data import DataLoader, Dataset
from pytorch_lightning import LightningDataModule, LightningModule
from matplotlib import pyplot as plt
from lhotse.dataset import BucketingSampler, OnTheFlyFeatures
from lhotse.dataset.collation import TokenCollater
from lhotse.recipes import download_librispeech, prepare_librispeech
from lhotse.dataset.vis import plot_batch
from lhotse import CutSet, RecordingSet, SupervisionSet, Fbank, FbankConfig
from pathlib import Path
from pprint import pprint

## Text-To-Speech

### Base Class

In [ ]:
#| export
class TTSDataset(Dataset):
    def __init__(self,
        tokenizer:TokenCollater, # text tokenizer
        num_mel_bins:int=80 # number of mel spectrogram bins
        ):
        self.extractor = OnTheFlyFeatures(Fbank(FbankConfig(num_mel_bins=num_mel_bins)))
        self.tokenizer = tokenizer

    def __getitem__(self, cuts: CutSet) -> dict:
        cuts = cuts.sort_by_duration()
        feats, feat_lens = self.extractor(cuts)
        tokens, token_lens = self.tokenizer(cuts)
        return {"feats_pad": feats, "ilens": feat_lens, "tokens_pad": tokens}


### LJSpeech DataModule

In [ ]:
# #| export
# class LJSpeechDataModule(LightningDataModule):
#     def __init__(self,
#         target_dir="/data/en", # where data will be saved / retrieved
#         dataset_parts="mini_librispeech", # either full librispeech or mini subset
#         output_dir="../recipes/tts/ljspeech/data" # where to save manifest
#     ):
#         super().__init__()
#         self.save_hyperparameters(logger=False)

#     def prepare_data(self,) -> None:
#         download_librispeech(target_dir=self.hparams.target_dir, dataset_parts=self.hparams.dataset_parts)

#     def setup(self, stage = None):
#         libri = prepare_librispeech(corpus_dir=Path(self.hparams.target_dir) / "LibriSpeech", output_dir=self.hparams.output_dir)
#         self.cuts_train = CutSet.from_manifests(**libri["train-clean-5"])
#         self.cuts_test = CutSet.from_manifests(**libri["dev-clean-2"])
#         self.tokenizer = TokenCollater(self.cuts_train)
#         self.tokenizer(self.cuts_test.subset(first=2))
#         self.tokenizer.inverse(*self.tokenizer(self.cuts_test.subset(first=2)))

#     def train_dataloader(self):
#         train_sampler = BucketingSampler(self.cuts_train, max_duration=300, shuffle=True, bucket_method="equal_duration")
#         return DataLoader(STTDataset(self.tokenizer), sampler=train_sampler, batch_size=None, num_workers=2)

#     def test_dataloader(self):
#         test_sampler = BucketingSampler(self.cuts_test, max_duration=400, shuffle=False, bucket_method="equal_duration")
#         return DataLoader(STTDataset(self.tokenizer), sampler=test_sampler, batch_size=None, num_workers=2)

#     @property
#     def model_kwargs(self):
#         return {
#             "odim": len(self.tokenizer.idx2token),
#         }

### LibriTTS DataModule

In [ ]:
#| export
from lhotse.recipes import download_libritts, prepare_libritts
from nimrod.text.tokenizers import Tokenizer
from nimrod.audio.embedding import EncoDec

In [ ]:
#| export
class LibriTTSDataModule(LightningDataModule):
    def __init__(self,
        target_dir="/data/en/libriTTS", # where data will be saved / retrieved
        dataset_parts=["dev-clean", "test-clean"], # either full libritts or subset
        output_dir="/home/syl20/slg/nimrod/recipes/libritts/data", # where to save manifest
        num_jobs=1 # num_jobs depending on number of cpus available
    ):
        super().__init__()
        self.save_hyperparameters(logger=False)

    def prepare_data(self,) -> None:
        # takes a while to download from openslr mirror (~15 min each for test/dev-clean)
        download_libritts(target_dir=self.hparams.target_dir, dataset_parts=self.hparams.dataset_parts)

    def setup(self, stage = None):
        self.libri = prepare_libritts(corpus_dir=Path(self.hparams.target_dir) / "LibriTTS", output_dir=self.hparams.output_dir, num_jobs=self.hparams.num_jobs)
        if stage == 'fit' or stage == None:
            self.cuts_train = CutSet.from_manifests(**self.libri["dev-clean"])
            self.cuts_test = CutSet.from_manifests(**self.libri["test-clean"])
            self.tokenizer = TokenCollater(self.cuts_train)
            self.tokenizer(self.cuts_test.subset(first=2))
            self.tokenizer.inverse(*self.tokenizer(self.cuts_test.subset(first=2)))
        if stage == "test":
            self.cuts_test = CutSet.from_manifests(**self.libri["test-clean"])
            self.tokenizer = TokenCollater(self.cuts_test)
            self.tokenizer(self.cuts_test.subset(first=2))
            self.tokenizer.inverse(*self.tokenizer(self.cuts_test.subset(first=2)))

    def train_dataloader(self):
        train_sampler = BucketingSampler(self.cuts_train, max_duration=300, shuffle=True, bucket_method="equal_duration")
        return DataLoader(TTSDataset(self.tokenizer), sampler=train_sampler, batch_size=None, num_workers=100)

    def test_dataloader(self):
        test_sampler = BucketingSampler(self.cuts_test, max_duration=400, shuffle=False, bucket_method="equal_duration")
        return DataLoader(TTSDataset(self.tokenizer), sampler=test_sampler, batch_size=None, num_workers=2)

    @property
    def model_kwargs(self):
        return {
            "odim": len(self.tokenizer.idx2token),
        }

### Usage

In [ ]:
dm = LibriTTSDataModule(
    target_dir="../data/en", 
    dataset_parts="test-clean",
    output_dir="../data/en/LibriTTS/test-clean",
    num_jobs=1
)

In [ ]:
# skip download and use local data folder
# dm.prepare_data()

In [ ]:
dm.setup(stage='test')

Scanning audio files (*.wav): 0it [00:00, ?it/s]00:00<?, ?it/s]
Scanning audio files (*.wav): 0it [00:00, ?it/s]
Scanning audio files (*.wav): 95it [00:00, 4875.90it/s]
Scanning audio files (*.wav): 0it [00:00, ?it/s]
Scanning audio files (*.wav): 0it [00:00, ?it/s]
Scanning audio files (*.wav): 0it [00:00, ?it/s]00:00<00:00, 48.20it/s]
Scanning audio files (*.wav): 0it [00:00, ?it/s]
Preparing LibriTTS parts: 100%|██████████| 7/7 [00:00<00:00, 50.96it/s]


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()